In [1]:
import pandas as pd
import numpy as np
import os
import glob

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
WIN = 'D:/'
MAC = '/Volumes/Transcend2/17_20_PhD/19_20_PhD_Metabolomics/'

osp = MAC

## Load in limma results

Data was obtained in Step 1/6.PeakSetFiltering

In [9]:
output_dir = osp+'/pymz/mzmine/peak_picked_files/samples_alignment/'
limma_filename = 'limma_independent_toptable_features.csv'


## Run mummichog

In [16]:
keyword = 'mm071020'
os.system('mummichog -f'+ output_dir + limma_filename +
          ' -m positive -u 3 -k MummichogResults -o '+ keyword+' -c 0.05')

0

## Annotate data

In [34]:
for f in os.listdir('MummichogResults'):
    if keyword in f:
        mummichog_folder = f

In [36]:
compounds = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/ListOfEmpiricalCompounds.tsv', sep='\t')
featureoutput = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/userInputData.txt', sep='\t')
pathway = pd.read_csv('MummichogResults/'+mummichog_folder+'/tables/mcg_pathwayanalysis_'+keyword+'.tsv', sep='\t')

In [60]:
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    matching_ids[key] = value

In [61]:
compounds_dict = {}
for _,row in compounds.iterrows():
    ids = row['massfeature_rows']
    ions = row['str_row_ion']
    compoundlist = row['compound_names']
    kegg_id = row['compounds']
    
    
    ids_list = ids.split(';')
    ions_list = ions.split(';')
    for i in range(len(ids_list)):
        compounds_dict[matching_ids[ids_list[i]]] = []
        compounds_dict[matching_ids[ids_list[i]]].append(ions_list[i].split('_')[1])
        compounds_dict[matching_ids[ids_list[i]]].append(compoundlist)
        compounds_dict[matching_ids[ids_list[i]]].append(kegg_id)
        

In [62]:
ecomp_dict = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    massfeature_rows = row['massfeature_rows'].split(';')
    for mass in massfeature_rows:
        if mass in ecomp_dict:
            ecomp_dict[mass].append(eid)
        else:
            ecomp_dict[mass] = []
            ecomp_dict[mass].append(eid)
            
cmpd_names = {}
for _,row in compounds.iterrows():
    eid = row['EID']
    name = row['compound_names']
    cmpd_names[eid] = name
matching_ids = {}
for _,row in featureoutput.iterrows():
    key = row['massfeature_rows']
    value = row['CompoundID_from_user']
    
    matching_ids[value] = key
for n in matching_ids:
    if matching_ids[n] in ecomp_dict:
        matching_ids[n] = ecomp_dict[matching_ids[n]]

id_names = {}
for i in matching_ids:
    if type(matching_ids[i]) is list:
        for f in matching_ids[i]:
            if f in cmpd_names:
                if i in id_names:
                    id_names[i].append(cmpd_names[f])
                else:
                    id_names[i] = []
                    id_names[i].append(cmpd_names[f])
matching_ids2 = {}
for n in matching_ids:
    
    if type(matching_ids[n]) is list:
        lista = matching_ids[n]
        for p in lista:
            if p in matching_ids2:
                matching_ids2[p].append(n)
            else:
                matching_ids2[p] = []
                matching_ids2[p].append(n)
pathway_dict = {}
for _,row in pathway.iterrows():
    
    pname = row['pathway']
    idlist =row['overlap_EmpiricalCompounds (id)']
    
    if type(idlist) != float:
        idlist = idlist.split(',')
        for p in idlist:
            
            if pname in pathway_dict:
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
            else:
                pathway_dict[pname] = []
                if p in matching_ids2:
                    pathway_dict[pname].append(matching_ids2[p]) 
                
pathway_dict2 = {}
for p in pathway_dict:
    
    for n in pathway_dict[p]:
        for i in n:
            if i in pathway_dict2:
                if p not in  pathway_dict2[i]:
                    pathway_dict2[i].append(p)
            else:
                pathway_dict2[i] = []
                pathway_dict2[i].append(p)
                

In [196]:
data = pd.read_csv(output_dir+limma_filename, sep='\t')

In [197]:
data.index = data['custom_id']

In [198]:
data['mm_compound_names'] = str(np.zeros(len(data)))
data['mm_pathway'] = str(np.zeros(len(data)))
data['mm_kegg_id'] = str(np.zeros(len(data)))
for row_id,row in data.iterrows():
    if row_id in id_names:
        data.at[row_id,'mm_compound_names'] = [str(name).lower() for name in id_names[row_id]]
    else:
        data.at[row_id,'mm_compound_names'] = 'NaN'
    
    if row_id in pathway_dict2:
        data.at[row_id,'mm_pathway'] = pathway_dict2[row_id]
    else:
        data.at[row_id,'mm_pathway'] = 'NaN'
    if row_id in compounds_dict:
        data.at[row_id,'mm_kegg_id'] = compounds_dict[row_id][2]
    else:
        data.at[row_id,'mm_kegg_id'] = 'NaN'
    
    

In [199]:
data

,row m/z,row retention time,adj-p,t-val,custom_id,mm_compound_names,mm_pathway,mm_kegg_id
custom_id,,,,,,,,
633,148.073458,11.915377,1.707271e-07,-7.230248,633,[(r)-mevalonate],[Squalene and cholesterol biosynthesis],C00418
613,147.076407,11.905214,1.707271e-07,-7.050995,613,[l-glutamine$3-ureidoisobutyrate$d-glutamine; ...,"[Nitrogen metabolism, Alanine and Aspartate Me...",C00906
3109,905.593771,2.998939,1.707271e-07,-7.048608,3109,NaN,NaN,NaN
2989,810.600011,3.171275,3.354703e-07,-6.773560,2989,NaN,NaN,NaN
324,126.021981,12.209536,3.354703e-07,-6.770043,324,[taurine],"[Methionine and cysteine metabolism, Bile acid...",C00245
...,...,...,...,...,...,...,...,...
1982,257.113047,8.046586,9.784447e-01,-0.035265,1982,NaN,NaN,NaN
240,118.086244,9.607425,9.792079e-01,-0.032271,240,[6-amino-2-oxohexanoate; 2-oxo-6-aminocaproate...,NaN,C00956
2037,264.180478,3.470832,9.818751e-01,0.026886,2037,NaN,NaN,NaN


## Matching with standards ToxID annotation

In [200]:
import pickle
with open(osp+'Metabolomics_datasets/standards_list.dict', 'rb') as file:
    standards_dict = pickle.load(file)

In [218]:
data['std_match'] = 'NaN'
ppm = 0.1
ppm_rt = 60
for rowid,row in data.iterrows():
    mz = row['row m/z']
    rt = row['row retention time']
    for key in standards_dict.keys():
        if mz >= key-ppm and mz <= key + ppm:
            if rt*60 >= standards_dict[key][1]-ppm_rt and rt*60 <= standards_dict[key][1]+ppm_rt:
                data.at[rowid, 'std_match'] = standards_dict[key][0]
        
                
       

In [224]:
peakset = {}
for _, row in data.iterrows():
    cid = row['custom_id']
    mz = row['row m/z']
    rt = row['row retention time']
    pval = row['adj-p']
    tval = row['t-val']
    mm_name = row['mm_compound_names']
    mm_path = row['mm_pathway']
    mm_kegg = row['mm_kegg_id']
    std = row['std_match']
    
    peakset[cid] = (mz, rt*60, pval, tval, mm_name, mm_path, mm_kegg, std)

In [227]:
with open(osp+'pymz/mzmine/peak_picked_files/filtered_peaksets.dict', 'wb') as handle:
    pickle.dump(peakset, handle)

## Check accuracy of mummichog annotation

In [220]:
notannotated = 0
annotated = 0
notannotateds = 0
annotateds = 0
match = 0
matchs =0
for i,row in data.iterrows():
    cmp = str(row['mm_compound_names']).lower()
    std = str(row['std_match']).lower()
    
    if cmp in ['nan']:
        notannotated +=1
        if not std in ['nan']:
            matchs +=1
           
    else:
        annotated +=1
        if not std in ['nan']:
            if std in cmp:
                match +=1
            else:
                print(row)
    
    if std in ['nan']:
        notannotateds +=1
    else:
        annotateds +=1
        
percentage_exact_matches = match*100/annotateds
percentage_not_annotated = matchs*100/annotateds
percentage_not_matched = (annotateds-match-matchs)*100/annotateds

print("Total number of annotated compounds using mummichog: {}".format(annotated))
print("Total number of annotated compounds using ToxID standards: {}".format(annotateds))
print("Total number of annotation matches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(match, percentage_exact_matches))
print("Total number of annotation mismatches between mummichog and ToxID: {0}, i.e. {1:.2f}%".format(annotateds-match-matchs, percentage_not_matched))
print("Total number of metabolites annotated with ToxID, but not with mummichog: {0}, i.e. {1:.2f}%".format(matchs, percentage_not_annotated))

row m/z                                               148.073
row retention time                                    11.9154
adj-p                                             1.70727e-07
t-val                                                -7.23025
custom_id                                                 633
mm_compound_names                            [(r)-mevalonate]
mm_pathway            [Squalene and cholesterol biosynthesis]
mm_kegg_id                                             C00418
std_match                                   o-acetyl-l-serine
Name: 633, dtype: object
row m/z                                                         136.043
row retention time                                              10.0993
adj-p                                                       0.000105625
t-val                                                          -4.78319
custom_id                                                           459
mm_compound_names                                      [l